In [14]:
import urllib.request, json 
import numpy as np
import cv2
import math
# with urllib.request.urlopen("https://a.windbornesystems.com/treasure/00.json") as url:
#     data = json.load(url)
# data = np.array(data)

# Exploration
Looking at the means of the data can give us an idea of what it actually is.  Since you told us they are locations, I will start by assuming they are latitudes and longitudes.  

Let's try mapping these locations on a map.  

In [3]:
data.mean(axis=0)

array([nan, nan, nan])

The first two indexes' means are around 0 mean they are the lat long while the 3rd element is positive suggesting it is height.  

In [15]:
def loc2pixels(location, zoom):
    lat = location[0]
    long = location[1]
    lat_rad = math.radians(lat)
    map_size = 256 * 2**zoom
    x = math.floor((long + 180) / 360 * map_size)
    y = math.floor((1 - math.log(math.tan(lat_rad) + 1 / math.cos(lat_rad)) / math.pi) / 2 * map_size)
    return x, y





In [37]:
def pin(image, location, alpha):
    color = (0, 0, 255)
    radius = 5
    thickness = -1  
    overlay = image.copy()
    cv2.circle(overlay, location, radius, color, thickness)
    return cv2.addWeighted(overlay, alpha, image, 1 - alpha, 0)
    

# Retrieving data

The "ill" formated jsons just don't have a starting bracket so formatting them is easy.

The first try is to see if the url is valid
The second is to check if the data is formatted correctly

This is still exploration so I won't clean it up but in the main code it will be cleaned up more.

In [ ]:
def get_data(day):
    if day < 10 and day >= 0:
        url = f"https://a.windbornesystems.com/treasure/0{day}.json"
    elif day < 24:
        url = f"https://a.windbornesystems.com/treasure/{day}.json"
    else:
        raise IndexError
    
    try:
        with urllib.request.urlopen(url) as response:
            raw_data = response.read().decode("utf-8", errors="ignore")
            if raw_data[0] != "[":
                raw_data = "[" + raw_data 
            try:
                 data = json.loads(raw_data)
            except:
                # This was for looking at the bad data

                # with open(f"day_{day}.txt", "wb") as f:
                #     f.write(raw_data)
                # print(response)
                # print(raw_data)
                print(f"({day}) Invalid JSON{url}")
                return None
    except Exception as e:
        print(f"({day}) {e}")
        return None
    print(f"({day}) Succeeded")
    return np.array(data)

In [12]:
datas = []
for day in range(24):
    datas.append(get_data(day))


(0) Succeeded
(1) Succeeded
(2) Succeeded
(3) Succeeded
(4) Succeeded
(5) HTTP Error 404: Not Found
(6) Succeeded
(7) Succeeded
(8) Succeeded
(9) Succeeded
(10) HTTP Error 404: Not Found
(11) Succeeded
(12) HTTP Error 404: Not Found
(13) Succeeded
(14) HTTP Error 404: Not Found
(15) HTTP Error 404: Not Found
(16) HTTP Error 404: Not Found
(17) Succeeded
(18) HTTP Error 404: Not Found
(19) Succeeded
(20) HTTP Error 404: Not Found
(21) HTTP Error 404: Not Found
(22) Succeeded
(23) HTTP Error 404: Not Found


In [26]:
image = cv2.imread("map.jpg")
zoom = math.log2(image.shape[1]/256)
historical_data = np.zeros((24, 1000, 3))
invalid_days = []
for day in range(24):
    data = datas[day]
    if data is None:
        invalid_days.append(day)
        continue
    for satellite_num in range(data.shape[0]):
        location = data[satellite_num]
        if np.isnan(location[0]) or np.isnan(location[1]):
            continue
        point = loc2pixels((location[0], location[1]), zoom)
        historical_data[day][satellite_num] = point[0], point[1], location[2]


In [63]:

filepath = "gif/"
filenames = []
count = 0
for max_day in range(24):
    filename = filepath + str(max_day)
    if max_day in invalid_days:
        continue
    image = cv2.imread("map.jpg")
    result = image.copy()
    print(max_day)
    for day in range(max_day+1):
        if day in invalid_days:
            continue
        for satellite_num in range(1000):
            location = int(historical_data[day][satellite_num][0]), int(historical_data[day][satellite_num][1])
            alpha = math.exp(-.2*(max_day-day)) # Exponential decay because it was getting too confusing
            result = pin(result, location, alpha) 
    cv2.imwrite(f"{filename}.png", result) 
            

0
1
2
3
4
6
7
8
9
11
13
17
19
22


In [61]:
import imageio
images = []
filenames.sort(key=lambda x: int(x.split('.')[0]))
for filename in filenames:
    images.append(imageio.imread(filepath + filename))

imageio.mimsave("movie.gif", images, duration = 0.1)

/var/folders/0p/q7n443n563d5v8gs7ft242tw0000gn/T/ipykernel_97204/2707566019.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filepath + filename))


# Final Thoughts
This was a good exploration of the data.

I think the bulk of my project will be estimating the missing data and day ahead data.  We can do this by taking wind data from a different source and then do some estimation from there.


I didn't use the height of the balloons in this exploration but it is as simple as changing the color of the dot.  

['0.png',
 '1.png',
 '2.png',
 '3.png',
 '4.png',
 '6.png',
 '7.png',
 '8.png',
 '9.png',
 '11.png',
 '13.png',
 '17.png',
 '19.png',
 '22.png']